# Install necessary libraries

In [ ]:
!pip install psycopg2-binary python-dotenv

# Import necessary libraries

In [8]:
import os
import psycopg2
import csv
from typing import List
from dotenv import load_dotenv
from datetime import datetime

# Defining functions

### Selective column export

In [9]:
def load_db_config() -> dict:
    """Load database configuration from environment variables."""
    load_dotenv()
    return {
        'dbname': os.getenv('DB_NAME'),
        'user': os.getenv('DB_USERNAME'),
        'password': os.getenv('DB_PASS'),
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT')
    }

def export_to_csv(columns: List[str], output_file: str = 'user_export.csv') -> None:
    """
    Export specified columns from the user table to a CSV file.
    
    Args:
        columns: List of column names to export
        output_file: Name of the output CSV file
    """
    try:
        # Connect to the database
        db_config = load_db_config()
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()
        
        # Create the SQL query with schema.table name
        columns_str = ', '.join(columns)
        query = f'SELECT {columns_str} FROM public.user'
        
        # Execute the query
        cursor.execute(query)
        
        # Write to CSV
        with open(output_file, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            # Write header
            writer.writerow(columns)
            # Write data
            for row in cursor:
                writer.writerow(row)
        
        print(f"Data successfully exported to {output_file}")
        
    except psycopg2.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

# Execution

In [10]:
if __name__ == "__main__":
    # Example usage
    columns_to_export = ["id", "email", "password", "role"]
    export_to_csv(columns_to_export)

Data successfully exported to user_export.csv


# Full table export

In [7]:
def load_db_config() -> dict:
    """Load database configuration from environment variables."""
    load_dotenv()
    return {
        'dbname': os.getenv('DB_NAME'),
        'user': os.getenv('DB_USERNAME'),
        'password': os.getenv('DB_PASS'),
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT')
    }

def export_full_table_to_csv():
    """Export the entire user table to a CSV file with timestamp in filename."""
    try:
        # Connect to the database
        db_config = load_db_config()
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()
        
        # Get column names
        cursor.execute("""
            SELECT column_name 
            FROM information_schema.columns 
            WHERE table_schema = 'public' 
            AND table_name = 'user'
            ORDER BY ordinal_position;
        """)
        columns = [col[0] for col in cursor.fetchall()]
        
        # Create filename with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'user_table_export_{timestamp}.csv'
        
        # Execute the query for all data
        cursor.execute('SELECT * FROM public.user')
        
        # Write to CSV
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            # Write header
            writer.writerow(columns)
            # Write data
            for row in cursor:
                writer.writerow(row)
        
        print(f"Full table successfully exported to {filename}")
        print(f"Total columns exported: {len(columns)}")
        print(f"Columns: {', '.join(columns)}")
        
    except psycopg2.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

if __name__ == "__main__":
    export_full_table_to_csv()

Full table successfully exported to user_table_export_20241230_220628.csv
Total columns exported: 28
Columns: id, email, password, role, isConfirmed, isActive, fullName, profilePicture, countryId, phone, additionalPhone, address01, address02, dateOfBirth, gender, createdAt, updatedAt, brandId, verificationToken, userId, isDeleted, isUserRegisterForBrand, passWordChangeRequest, isOwner, isBiometric, newPassword, contactPersonId, isBlocked
